# Chapter 8. Dimensionality Reduction

## PCA

Using Numpy's svd()

In [1]:
# extra code to create a 3D dataset

import numpy as np
from scipy.spatial.transform import Rotation

m = 60
X = np.zeros((m, 3))  # initialize 3D dataset
np.random.seed(42)
angles = (np.random.rand(m) ** 3 + 0.5) * 2 * np.pi  # uneven distribution
X[:, 0], X[:, 1] = np.cos(angles), np.sin(angles) * 0.5  # oval
X += 0.28 * np.random.randn(m, 3)  # add more noise
X = Rotation.from_rotvec([np.pi / 29, -np.pi / 20, np.pi / 4]).apply(X)
X += [0.2, 0, 0.2]  # shift a bit

In [2]:
import numpy as np

X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt[0]
c2 = Vt[1]

In [3]:
W2 = Vt[:2].T
X2D = X_centered @ W2

**Using Scikit-learn's PCA class**

In [4]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

In [5]:
pca.components_

array([[ 0.67857588,  0.70073508,  0.22023881],
       [ 0.72817329, -0.6811147 , -0.07646185]])

In [6]:
pca.explained_variance_ratio_

array([0.7578477 , 0.15186921])

the first PC had 76% variance and the 2nd, 15%. Meaning the 3rd PC had little information with 9% 

### Choosing the Right Number of Dimensions

In [7]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', as_frame=False)

# splitting the data
X_train, y_train = mnist.data[:60_000], mnist.target[:60_000]
X_test, y_test = mnist.data[60_000:], mnist.target[60_000:]

In [8]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

In [9]:
pca.n_components_

154

**Using dimensionality reduction as a preprocessing step for a supervised learning task (e.g., classification):**

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

In [11]:
## make pipeline
clf = make_pipeline((PCA(random_state=42)),
                   (RandomForestClassifier(random_state=42)))
# randomized cv 
param_distrib = {
    "pca__n_components": np.arange(2, 100),
    "randomforestclassifier__n_estimators": np.arange(30, 500)
}

random_search = RandomizedSearchCV(clf, param_distrib, n_iter=10,
                                  random_state=42)
# fit 
random_search.fit(X_train[:1000], y_train[:1000])

RandomizedSearchCV(estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(random_state=42))]),
                   param_distributions={'pca__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57...
       407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419,
       420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432,
       433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445,
       446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458,
       459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471,
       472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484,
       485, 486

In [12]:
random_search.best_params_

{'randomforestclassifier__n_estimators': 430, 'pca__n_components': 37}

In [13]:
# converting the reduced data to its original 
X_recovered = pca.inverse_transform(X_reduced)

## Randomized PCA

In [14]:
rnd_pca = PCA(n_components=154, svd_solver="randomized",
             random_state=42)
X_reduced = rnd_pca.fit_transform(X_train)


`svd_solver="randomized"` uses a stochastic algorithm called Randomized PCA that quickly finds an approximation of the first d principal components

## Incremental PCA

In [15]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100  # set no. of training batches
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
# incremental learning
X_reduced = inc_pca.transform(X_train)

In [16]:
from sklearn.random_projection import johnson_lindenstrauss_min_dim

m, epsilon = 5_000, 0.1
d = johnson_lindenstrauss_min_dim(m, eps=epsilon)
d  # reduces dimension to 7300

7300

In [17]:
n = 20_000
np.random.seed(42)
P = np.random.randn(d, n) / np.sqrt(d) # std dev = square root of variance

In [18]:
X = np.random.randn(m, n) # generate a fake dataset
X_reduced = X @ P.T

In [19]:
X_reduced

array([[ 0.65601016, -1.52157349, -2.13016582, ..., -3.30298058,
         1.86509223, -0.62205374],
       [-1.15983875, -1.31973727,  0.13553982, ..., -0.9461484 ,
        -0.01875402,  1.15834454],
       [-2.79258363,  0.47631317, -3.2293912 , ...,  1.41899815,
         3.03817514, -2.07598188],
       ...,
       [ 0.18867284,  0.58379053,  1.42444552, ...,  0.13507484,
         1.83874287, -0.39265772],
       [ 2.54718434,  0.32063561, -3.59830134, ...,  0.49247486,
         0.46798188,  0.09910125],
       [ 3.68422382, -2.65487692,  0.55763148, ...,  2.14734207,
        -1.82840766,  0.76915154]])

Scikit-Learn offers a `GaussianRandomProjection` class

In [20]:
from sklearn.random_projection import GaussianRandomProjection

gaussian_rnd_proj = GaussianRandomProjection(eps=epsilon, random_state=42)
X_reduced = gaussian_rnd_proj.fit_transform(X)
X_reduced

array([[ 0.65601016, -1.52157349, -2.13016582, ..., -3.30298058,
         1.86509223, -0.62205374],
       [-1.15983875, -1.31973727,  0.13553982, ..., -0.9461484 ,
        -0.01875402,  1.15834454],
       [-2.79258363,  0.47631317, -3.2293912 , ...,  1.41899815,
         3.03817514, -2.07598188],
       ...,
       [ 0.18867284,  0.58379053,  1.42444552, ...,  0.13507484,
         1.83874287, -0.39265772],
       [ 2.54718434,  0.32063561, -3.59830134, ...,  0.49247486,
         0.46798188,  0.09910125],
       [ 3.68422382, -2.65487692,  0.55763148, ...,  2.14734207,
        -1.82840766,  0.76915154]])

In [21]:
# inverse transformation
components_pinv = np.linalg.pinv(gaussian_rnd_proj.components_)
X_recovered = X_reduced @ components_pinv.T

In [22]:
X_recovered 

array([[-0.78285178,  0.47707426,  0.6955112 , ...,  0.98188064,
         0.87441748, -0.71906857],
       [ 0.97744839, -0.14541327, -0.3545605 , ..., -0.18058073,
         0.32781621,  0.81330977],
       [ 0.20082204, -0.05543369, -1.29965877, ..., -0.81101979,
         0.40761253, -0.61037469],
       ...,
       [ 0.91281977, -1.12843126,  0.41045219, ..., -0.07550752,
         0.17787192,  0.1177291 ],
       [ 0.8433102 , -0.78806083,  0.76058445, ..., -0.49409368,
        -0.08168829,  0.02277401],
       [-0.11412415, -0.30551047,  0.97617111, ..., -0.25285318,
        -0.56290596,  0.38087206]])

## Locally Linear Embedding (LLE)